In [9]:
import sys

In [10]:
!{sys.executable} -m pip install -r ../requirements.txt

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Defaulting to user installation because normal site-packages is not writeable


In [11]:
import struct
import zlib
import array

class PNGReader:    
    def __init__(self, filename):
        self.filename = filename
        self.width = 0
        self.height = 0
        self.bit_depth = 0
        self.color_type = 0
        self.compression_method = 0
        self.filter_method = 0
        self.interlace_method = 0
        self.image_data = b''
        self.pixels = []
    
    def read_png(self):
        """Read and parse the PNG file"""
        with open(self.filename, 'rb') as f:
            # Check PNG signature
            signature = f.read(8)
            if signature != b'\x89PNG\r\n\x1a\n':
                raise ValueError("Not a valid PNG file")
            
            # Read chunks
            while True:
                chunk_data = f.read(4)
                if len(chunk_data) < 4:
                    break
                
                chunk_length = struct.unpack('>I', chunk_data)[0]
                chunk_type = f.read(4)
                chunk_data = f.read(chunk_length)
                chunk_crc = f.read(4)
                
                if chunk_type == b'IHDR':
                    self._parse_ihdr(chunk_data)
                elif chunk_type == b'IDAT':
                    self.image_data += chunk_data
                elif chunk_type == b'IEND':
                    break
            
            # Decompress and process image data
            self._process_image_data()
            return self.pixels
    
    def _parse_ihdr(self, data):
        """Parse the IHDR (Image Header) chunk"""
        if len(data) != 13:
            raise ValueError("Invalid IHDR chunk")
        
        (self.width, self.height, self.bit_depth, self.color_type,
         self.compression_method, self.filter_method, self.interlace_method) = struct.unpack('>IIBBBBB', data)
        
        print(f"Image dimensions: {self.width} x {self.height}")
        print(f"Bit depth: {self.bit_depth}")
        print(f"Color type: {self.color_type}")
    
    def _process_image_data(self):
        """Decompress and filter the image data"""
        # Decompress the image data
        decompressed_data = zlib.decompress(self.image_data)
        
        # Calculate bytes per pixel and scanline
        if self.color_type == 0:  # Grayscale
            bytes_per_pixel = self.bit_depth // 8 if self.bit_depth >= 8 else 1
        elif self.color_type == 2:  # RGB
            bytes_per_pixel = 3 * (self.bit_depth // 8)
        elif self.color_type == 6:  # RGBA
            bytes_per_pixel = 4 * (self.bit_depth // 8)
        else:
            raise ValueError(f"Unsupported color type: {self.color_type}")
        
        scanline_length = self.width * bytes_per_pixel + 1  # +1 for filter type
        
        # Process each scanline
        self.pixels = []
        for y in range(self.height):
            scanline_start = y * scanline_length
            filter_type = decompressed_data[scanline_start]
            scanline_data = decompressed_data[scanline_start + 1:scanline_start + scanline_length]
            
            # Apply defiltering
            if y == 0:
                previous_scanline = b'\x00' * (scanline_length - 1)
            else:
                previous_scanline = self.pixels[y - 1]
            
            filtered_scanline = self._defilter_scanline(scanline_data, previous_scanline, filter_type, bytes_per_pixel)
            self.pixels.append(filtered_scanline)
    
    def _defilter_scanline(self, scanline, previous_scanline, filter_type, bytes_per_pixel):
        """Apply PNG defiltering to a scanline"""
        if filter_type == 0:  # None
            return scanline
        elif filter_type == 1:  # Sub
            result = bytearray(scanline)
            for i in range(bytes_per_pixel, len(scanline)):
                result[i] = (result[i] + result[i - bytes_per_pixel]) % 256
            return bytes(result)
        elif filter_type == 2:  # Up
            result = bytearray()
            for i in range(len(scanline)):
                result.append((scanline[i] + previous_scanline[i]) % 256)
            return bytes(result)
        elif filter_type == 3:  # Average
            result = bytearray()
            for i in range(len(scanline)):
                left = result[i - bytes_per_pixel] if i >= bytes_per_pixel else 0
                up = previous_scanline[i]
                average = (left + up) // 2
                result.append((scanline[i] + average) % 256)
            return bytes(result)
        elif filter_type == 4:  # Paeth
            result = bytearray()
            for i in range(len(scanline)):
                left = result[i - bytes_per_pixel] if i >= bytes_per_pixel else 0
                up = previous_scanline[i]
                upper_left = previous_scanline[i - bytes_per_pixel] if i >= bytes_per_pixel else 0
                paeth = self._paeth_predictor(left, up, upper_left)
                result.append((scanline[i] + paeth) % 256)
            return bytes(result)
        else:
            raise ValueError(f"Unknown filter type: {filter_type}")
    
    def _paeth_predictor(self, a, b, c):
        """Paeth predictor algorithm for PNG filtering"""
        p = a + b - c
        pa = abs(p - a)
        pb = abs(p - b)
        pc = abs(p - c)
        
        if pa <= pb and pa <= pc:
            return a
        elif pb <= pc:
            return b
        else:
            return c
    
    def get_pixel_array(self):
        """Convert the pixel data to a 2D array"""
        pixel_array = []
        
        if self.color_type == 0:  # Grayscale
            for y in range(self.height):
                row = []
                for x in range(self.width):
                    if self.bit_depth == 8:
                        pixel = self.pixels[y][x]
                    else:  # bit_depth == 16
                        pixel = struct.unpack('>H', self.pixels[y][x*2:(x+1)*2])[0]
                    row.append(pixel)
                pixel_array.append(row)
        
        elif self.color_type == 2:  # RGB
            for y in range(self.height):
                row = []
                for x in range(self.width):
                    if self.bit_depth == 8:
                        r = self.pixels[y][x*3]
                        g = self.pixels[y][x*3 + 1]
                        b = self.pixels[y][x*3 + 2]
                        row.append((r, g, b))
                    else:  # bit_depth == 16
                        r = struct.unpack('>H', self.pixels[y][x*6:x*6+2])[0]
                        g = struct.unpack('>H', self.pixels[y][x*6+2:x*6+4])[0]
                        b = struct.unpack('>H', self.pixels[y][x*6+4:x*6+6])[0]
                        row.append((r, g, b))
                pixel_array.append(row)
        
        elif self.color_type == 6:  # RGBA
            for y in range(self.height):
                row = []
                for x in range(self.width):
                    if self.bit_depth == 8:
                        r = self.pixels[y][x*4]
                        g = self.pixels[y][x*4 + 1]
                        b = self.pixels[y][x*4 + 2]
                        a = self.pixels[y][x*4 + 3]
                        row.append((r, g, b, a))
                    else:  # bit_depth == 16
                        r = struct.unpack('>H', self.pixels[y][x*8:x*8+2])[0]
                        g = struct.unpack('>H', self.pixels[y][x*8+2:x*8+4])[0]
                        b = struct.unpack('>H', self.pixels[y][x*8+4:x*8+6])[0]
                        a = struct.unpack('>H', self.pixels[y][x*8+6:x*8+8])[0]
                        row.append((r, g, b, a))
                pixel_array.append(row)
        
        return pixel_array

# Example usage
def read_png_file(filename):
    """Simple function to read a PNG file and return pixel data"""
    reader = PNGReader(filename)
    pixels = reader.read_png()
    pixel_array = reader.get_pixel_array()
    
    print(f"Successfully read PNG: {filename}")
    print(f"Dimensions: {reader.width} x {reader.height}")
    print(f"Color channels: {reader.color_type}")
    
    return pixel_array, reader

In [12]:
# Read the PNG file
try:
    pixel_data, png_reader = read_png_file('../cat.png')
    
    # Display some information about the image
    print(f"Image loaded successfully!")
    print(f"Width: {png_reader.width}")
    print(f"Height: {png_reader.height}")
    print(f"Bit depth: {png_reader.bit_depth}")
    print(f"Color type: {png_reader.color_type}")
    
    # Show a few sample pixels
    print(f"\nFirst few pixels:")
    for i in range(min(3, len(pixel_data))):
        print(f"Row {i}: {pixel_data[i][:5]}...")  # Show first 5 pixels of each row
        
except FileNotFoundError:
    print("cat.png not found. Please make sure the file exists in the parent directory.")
except Exception as e:
    print(f"Error reading PNG: {e}")

Image dimensions: 400 x 400
Bit depth: 8
Color type: 6
Successfully read PNG: ../cat.png
Dimensions: 400 x 400
Color channels: 6
Image loaded successfully!
Width: 400
Height: 400
Bit depth: 8
Color type: 6

First few pixels:
Row 0: [(0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0)]...
Row 1: [(0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0)]...
Row 2: [(0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0)]...
Successfully read PNG: ../cat.png
Dimensions: 400 x 400
Color channels: 6
Image loaded successfully!
Width: 400
Height: 400
Bit depth: 8
Color type: 6

First few pixels:
Row 0: [(0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0)]...
Row 1: [(0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0)]...
Row 2: [(0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0)]...


In [13]:
# Convert RGB/RGBA to grayscale manually
def rgb_to_grayscale(pixel_data, color_type):
    grayscale_data = []
    
    for row in pixel_data:
        grayscale_row = []
        for pixel in row:
            if color_type == 0:  # Already grayscale
                grayscale_row.append(pixel)
            elif color_type == 2:  # RGB
                r, g, b = pixel
                # Use luminance formula: 0.299*R + 0.587*G + 0.114*B
                gray = int(0.299 * r + 0.587 * g + 0.114 * b)
                grayscale_row.append(gray)
            elif color_type == 6:  # RGBA
                r, g, b, a = pixel
                gray = int(0.299 * r + 0.587 * g + 0.114 * b)
                grayscale_row.append(gray)
        grayscale_data.append(grayscale_row)
    
    return grayscale_data

# Simple ASCII art representation
def display_as_ascii(pixel_data, width=50):
    ascii_chars = " .:-=+*#%@"
    
    # Calculate step size to fit desired width
    step_y = len(pixel_data) // (width // 2) if len(pixel_data) > width // 2 else 1
    step_x = len(pixel_data[0]) // width if len(pixel_data[0]) > width else 1
    
    ascii_art = []
    for y in range(0, len(pixel_data), step_y):
        line = ""
        for x in range(0, len(pixel_data[y]), step_x):
            # Normalize pixel value to ASCII char index
            if isinstance(pixel_data[y][x], tuple):
                # If it's RGB/RGBA, use red channel for simplicity
                pixel_value = pixel_data[y][x][0]
            else:
                pixel_value = pixel_data[y][x]
            
            # Map pixel value (0-255) to ASCII character
            char_index = min(len(ascii_chars) - 1, pixel_value * len(ascii_chars) // 256)
            line += ascii_chars[char_index]
        ascii_art.append(line)
    
    return ascii_art

if 'pixel_data' in globals() and pixel_data:
    # Convert to grayscale
    grayscale = rgb_to_grayscale(pixel_data, png_reader.color_type)
    print(f"\nGrayscale conversion completed!")
    print(f"Sample grayscale values from first row: {grayscale[0][:10]}...")
    
    # Display as ASCII art
    print(f"\nASCII representation of the image:")
    ascii_art = display_as_ascii(grayscale, width=60)
    for line in ascii_art[:20]:  # Show first 20 lines
        print(line)
    
    if len(ascii_art) > 20:
        print("... (truncated)")
else:
    print("No pixel data available. Please run the previous cell first.")


Grayscale conversion completed!
Sample grayscale values from first row: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...

ASCII representation of the image:
                                                                   
-------------*****%::::::::::::::::::::::::=##:::::::::::::::::::::
 .:::::::::::**###**#                   +###**.                    
  .::::::::::++*##*#*#======          ######+*...                  
   .::::::::.+++*#%%#+***+*+*+*+#***#**###+**                      
    .........#+=-=+=+++++++*++++*+++++****#+*.                     
      .......%#=:****++#*=*-*+=#=-++++#+=*#*:-.                    
         ...-#%*==+*###%%*#%--==%*+##**++=+#*-::.                  
++++++++++++-%%%#####*=.--#*#*+*%##*###***##***********************
 :---------- %#**=+*#*   -*@####@: * =*####%%%%%%%%%%%%%%%%%%%%%%%%
 .-----------#+++=++%+=*- *%##%%#.+ :=%%#+#%:======================
   :---------+-==*+****##=+=**#+++@@@%++***#::                     
             -*---=*==#*+**:=*=*%#%#+=***

In [14]:
def get_pixel_statistics(pixel_data, color_type):
    """Calculate basic statistics for the pixel data"""
    if color_type == 0:  # Grayscale
        values = [pixel for row in pixel_data for pixel in row]
        return {
            'min': min(values),
            'max': max(values),
            'mean': sum(values) / len(values),
            'total_pixels': len(values)
        }
    else:  # RGB or RGBA
        r_values = [pixel[0] for row in pixel_data for pixel in row]
        g_values = [pixel[1] for row in pixel_data for pixel in row]
        b_values = [pixel[2] for row in pixel_data for pixel in row]
        
        return {
            'red': {'min': min(r_values), 'max': max(r_values), 'mean': sum(r_values) / len(r_values)},
            'green': {'min': min(g_values), 'max': max(g_values), 'mean': sum(g_values) / len(g_values)},
            'blue': {'min': min(b_values), 'max': max(b_values), 'mean': sum(b_values) / len(b_values)},
            'total_pixels': len(r_values)
        }

def save_as_simple_format(pixel_data, filename, color_type):
    """Save pixel data as a simple text format for inspection"""
    with open(filename, 'w') as f:
        f.write(f"# Image data - Color type: {color_type}\n")
        f.write(f"# Dimensions: {len(pixel_data[0])} x {len(pixel_data)}\n")
        
        for y, row in enumerate(pixel_data):
            f.write(f"# Row {y}\n")
            for x, pixel in enumerate(row):
                f.write(f"{x},{y}: {pixel}\n")

def crop_image(pixel_data, x1, y1, x2, y2):
    """Crop the image to the specified rectangle"""
    cropped = []
    for y in range(y1, min(y2, len(pixel_data))):
        row = []
        for x in range(x1, min(x2, len(pixel_data[y]))):
            row.append(pixel_data[y][x])
        if row:  # Only add non-empty rows
            cropped.append(row)
    return cropped

if 'pixel_data' in globals() and pixel_data:
    # Get statistics
    stats = get_pixel_statistics(pixel_data, png_reader.color_type)
    print("Image Statistics:")
    print(f"Total pixels: {stats['total_pixels']}")
    
    if png_reader.color_type == 0:
        print(f"Grayscale - Min: {stats['min']}, Max: {stats['max']}, Mean: {stats['mean']:.2f}")
    else:
        print(f"Red   - Min: {stats['red']['min']}, Max: {stats['red']['max']}, Mean: {stats['red']['mean']:.2f}")
        print(f"Green - Min: {stats['green']['min']}, Max: {stats['green']['max']}, Mean: {stats['green']['mean']:.2f}")
        print(f"Blue  - Min: {stats['blue']['min']}, Max: {stats['blue']['max']}, Mean: {stats['blue']['mean']:.2f}")
    
    # Example crop (top-left 50x50 pixels)
    if len(pixel_data) > 50 and len(pixel_data[0]) > 50:
        cropped = crop_image(pixel_data, 0, 0, 50, 50)
        print(f"\nCropped a 50x50 section from top-left")
        print(f"Cropped dimensions: {len(cropped[0])} x {len(cropped)}")
    
    print(f"\nYou now have raw pixel data that you can manipulate without any external libraries!")
    print(f"pixel_data contains the raw RGB/RGBA values as a 2D list")
    print(f"Each pixel is accessible as: pixel_data[y][x]")
else:
    print("No pixel data available. Please run the PNG reading cell first.")

Image Statistics:
Total pixels: 160000
Red   - Min: 0, Max: 255, Mean: 127.36
Green - Min: 0, Max: 255, Mean: 99.77
Blue  - Min: 0, Max: 255, Mean: 76.03

Cropped a 50x50 section from top-left
Cropped dimensions: 50 x 50

You now have raw pixel data that you can manipulate without any external libraries!
pixel_data contains the raw RGB/RGBA values as a 2D list
Each pixel is accessible as: pixel_data[y][x]


In [15]:
import struct
import zlib
import array

class PNGWriter:
    """
    A pure Python PNG writer that saves pixel arrays as PNG files without external libraries.
    Supports RGB, RGBA, and grayscale formats.
    """
    
    def __init__(self, width, height, color_type=2, bit_depth=8):
        """
        Initialize PNG writer
        
        Args:
            width: Image width in pixels
            height: Image height in pixels
            color_type: 0=Grayscale, 2=RGB, 6=RGBA
            bit_depth: Bits per sample (8 or 16)
        """
        self.width = width
        self.height = height
        self.color_type = color_type
        self.bit_depth = bit_depth
        
        # Calculate bytes per pixel
        if color_type == 0:  # Grayscale
            self.bytes_per_pixel = bit_depth // 8
        elif color_type == 2:  # RGB
            self.bytes_per_pixel = 3 * (bit_depth // 8)
        elif color_type == 6:  # RGBA
            self.bytes_per_pixel = 4 * (bit_depth // 8)
        else:
            raise ValueError(f"Unsupported color type: {color_type}")
    
    def write_png(self, pixel_data, filename):
        """
        Write pixel data to a PNG file
        
        Args:
            pixel_data: 2D array of pixel values
            filename: Output PNG filename
        """
        with open(filename, 'wb') as f:
            # Write PNG signature
            f.write(b'\x89PNG\r\n\x1a\n')
            
            # Write IHDR chunk
            self._write_ihdr(f)
            
            # Prepare and write IDAT chunk
            compressed_data = self._prepare_image_data(pixel_data)
            self._write_idat(f, compressed_data)
            
            # Write IEND chunk
            self._write_iend(f)
        
        print(f"PNG file saved: {filename}")
        print(f"Dimensions: {self.width} x {self.height}")
        print(f"Color type: {self.color_type}, Bit depth: {self.bit_depth}")
    
    def _write_chunk(self, f, chunk_type, data):
        """Write a PNG chunk with length, type, data, and CRC"""
        length = len(data)
        f.write(struct.pack('>I', length))
        f.write(chunk_type)
        f.write(data)
        
        # Calculate CRC32 of type + data
        crc = zlib.crc32(chunk_type + data) & 0xffffffff
        f.write(struct.pack('>I', crc))
    
    def _write_ihdr(self, f):
        """Write the IHDR (Image Header) chunk"""
        ihdr_data = struct.pack('>IIBBBBB',
                               self.width,
                               self.height,
                               self.bit_depth,
                               self.color_type,
                               0,  # compression method
                               0,  # filter method
                               0)  # interlace method
        self._write_chunk(f, b'IHDR', ihdr_data)
    
    def _write_idat(self, f, compressed_data):
        """Write the IDAT (Image Data) chunk"""
        self._write_chunk(f, b'IDAT', compressed_data)
    
    def _write_iend(self, f):
        """Write the IEND chunk"""
        self._write_chunk(f, b'IEND', b'')
    
    def _prepare_image_data(self, pixel_data):
        """Convert pixel data to PNG format and compress it"""
        raw_data = bytearray()
        
        for y in range(self.height):
            # Add filter type (0 = None filter)
            raw_data.append(0)
            
            # Add pixel data for this scanline
            for x in range(self.width):
                pixel = pixel_data[y][x]
                
                if self.color_type == 0:  # Grayscale
                    if isinstance(pixel, (tuple, list)):
                        # If RGB/RGBA tuple provided, convert to grayscale
                        gray = int(0.299 * pixel[0] + 0.587 * pixel[1] + 0.114 * pixel[2])
                    else:
                        gray = int(pixel)
                    
                    if self.bit_depth == 8:
                        raw_data.append(gray & 0xFF)
                    else:  # 16-bit
                        raw_data.extend(struct.pack('>H', gray & 0xFFFF))
                
                elif self.color_type == 2:  # RGB
                    if isinstance(pixel, (tuple, list)) and len(pixel) >= 3:
                        r, g, b = pixel[0], pixel[1], pixel[2]
                    else:
                        # If single value provided, use as grayscale
                        r = g = b = int(pixel)
                    
                    if self.bit_depth == 8:
                        raw_data.extend([r & 0xFF, g & 0xFF, b & 0xFF])
                    else:  # 16-bit
                        raw_data.extend(struct.pack('>HHH', r & 0xFFFF, g & 0xFFFF, b & 0xFFFF))
                
                elif self.color_type == 6:  # RGBA
                    if isinstance(pixel, (tuple, list)) and len(pixel) >= 4:
                        r, g, b, a = pixel[0], pixel[1], pixel[2], pixel[3]
                    elif isinstance(pixel, (tuple, list)) and len(pixel) == 3:
                        r, g, b, a = pixel[0], pixel[1], pixel[2], 255
                    else:
                        # If single value provided, use as grayscale with full alpha
                        r = g = b = int(pixel)
                        a = 255
                    
                    if self.bit_depth == 8:
                        raw_data.extend([r & 0xFF, g & 0xFF, b & 0xFF, a & 0xFF])
                    else:  # 16-bit
                        raw_data.extend(struct.pack('>HHHH', r & 0xFFFF, g & 0xFFFF, b & 0xFFFF, a & 0xFFFF))
        
        # Compress the raw data
        compressed_data = zlib.compress(bytes(raw_data))
        return compressed_data

# Convenience functions for different image types
def write_png_rgb(pixel_data, filename, bit_depth=8):
    """Write RGB pixel data to PNG file"""
    height = len(pixel_data)
    width = len(pixel_data[0]) if height > 0 else 0
    
    writer = PNGWriter(width, height, color_type=2, bit_depth=bit_depth)
    writer.write_png(pixel_data, filename)

def write_png_rgba(pixel_data, filename, bit_depth=8):
    """Write RGBA pixel data to PNG file"""
    height = len(pixel_data)
    width = len(pixel_data[0]) if height > 0 else 0
    
    writer = PNGWriter(width, height, color_type=6, bit_depth=bit_depth)
    writer.write_png(pixel_data, filename)

def write_png_grayscale(pixel_data, filename, bit_depth=8):
    """Write grayscale pixel data to PNG file"""
    height = len(pixel_data)
    width = len(pixel_data[0]) if height > 0 else 0
    
    writer = PNGWriter(width, height, color_type=0, bit_depth=bit_depth)
    writer.write_png(pixel_data, filename)

print("PNG Writer classes and functions loaded!")
print("Available functions:")
print("- write_png_rgb(pixel_data, filename)")
print("- write_png_rgba(pixel_data, filename)")  
print("- write_png_grayscale(pixel_data, filename)")
print("- PNGWriter class for advanced usage")

PNG Writer classes and functions loaded!
Available functions:
- write_png_rgb(pixel_data, filename)
- write_png_rgba(pixel_data, filename)
- write_png_grayscale(pixel_data, filename)
- PNGWriter class for advanced usage


In [16]:
# Demonstrate PNG writing with the loaded image data
if 'pixel_data' in globals() and pixel_data and 'grayscale' in globals():
    print("=== PNG Writing Demonstrations ===\n")
    
    # 1. Save original image as PNG
    print("1. Saving original image...")
    write_png_rgb(pixel_data, "output_original.png")
    
    # 2. Save grayscale version
    print("\n2. Saving grayscale version...")
    write_png_grayscale(grayscale, "output_grayscale.png")
    
    # 3. Create a simple test image and save it
    print("\n3. Creating and saving a test gradient image...")
    test_width, test_height = 100, 100
    gradient_image = []
    
    for y in range(test_height):
        row = []
        for x in range(test_width):
            # Create a diagonal gradient
            red = int((x / test_width) * 255)
            green = int((y / test_height) * 255)
            blue = int(((x + y) / (test_width + test_height)) * 255)
            row.append((red, green, blue))
        gradient_image.append(row)
    
    write_png_rgb(gradient_image, "output_gradient.png")
    
    # 4. Create and save a simple pattern with transparency
    print("\n4. Creating RGBA image with transparency...")
    pattern_size = 80
    pattern_image = []
    
    for y in range(pattern_size):
        row = []
        for x in range(pattern_size):
            # Create a checkerboard pattern with transparency
            checker = ((x // 10) + (y // 10)) % 2
            if checker:
                # White with varying transparency
                alpha = int((x / pattern_size) * 255)
                row.append((255, 255, 255, alpha))
            else:
                # Black, fully opaque
                row.append((0, 0, 0, 255))
        pattern_image.append(row)
    
    write_png_rgba(pattern_image, "output_pattern_rgba.png")
    
    # 5. Crop and save a section of the original image
    if len(pixel_data) > 100 and len(pixel_data[0]) > 100:
        print("\n5. Saving cropped section...")
        cropped_section = crop_image(pixel_data, 50, 50, 150, 150)
        write_png_rgb(cropped_section, "output_cropped.png")
    
    print("\n=== All PNG files saved successfully! ===")
    print("Files created:")
    print("- output_original.png (original image)")
    print("- output_grayscale.png (grayscale version)")
    print("- output_gradient.png (test gradient)")
    print("- output_pattern_rgba.png (RGBA pattern)")
    if len(pixel_data) > 100 and len(pixel_data[0]) > 100:
        print("- output_cropped.png (cropped section)")

else:
    print("No pixel data available. Please run the PNG reading cells first to load an image.")
    print("\nHowever, you can still create and save new images:")
    print("Example:")
    print("# Create a simple 50x50 red square")
    print("red_square = [[(255, 0, 0) for x in range(50)] for y in range(50)]")
    print("write_png_rgb(red_square, 'red_square.png')")

=== PNG Writing Demonstrations ===

1. Saving original image...
PNG file saved: output_original.png
Dimensions: 400 x 400
Color type: 2, Bit depth: 8

2. Saving grayscale version...
PNG file saved: output_grayscale.png
Dimensions: 400 x 400
Color type: 0, Bit depth: 8

3. Creating and saving a test gradient image...
PNG file saved: output_gradient.png
Dimensions: 100 x 100
Color type: 2, Bit depth: 8

4. Creating RGBA image with transparency...
PNG file saved: output_pattern_rgba.png
Dimensions: 80 x 80
Color type: 6, Bit depth: 8

5. Saving cropped section...
PNG file saved: output_cropped.png
Dimensions: 100 x 100
Color type: 2, Bit depth: 8

=== All PNG files saved successfully! ===
Files created:
- output_original.png (original image)
- output_grayscale.png (grayscale version)
- output_gradient.png (test gradient)
- output_pattern_rgba.png (RGBA pattern)
- output_cropped.png (cropped section)
PNG file saved: output_pattern_rgba.png
Dimensions: 80 x 80
Color type: 6, Bit depth: 8

5

In [17]:
# Advanced PNG writing utilities and examples

def create_solid_color_image(width, height, color):
    """Create a solid color image"""
    if isinstance(color, (tuple, list)):
        return [[color for x in range(width)] for y in range(height)]
    else:
        return [[color for x in range(width)] for y in range(height)]

def create_gradient_image(width, height, start_color, end_color, direction='horizontal'):
    """Create a gradient image between two colors"""
    image = []
    
    for y in range(height):
        row = []
        for x in range(width):
            if direction == 'horizontal':
                ratio = x / (width - 1) if width > 1 else 0
            elif direction == 'vertical':
                ratio = y / (height - 1) if height > 1 else 0
            elif direction == 'diagonal':
                ratio = (x + y) / (width + height - 2) if (width + height) > 2 else 0
            else:
                ratio = 0
            
            # Interpolate between start and end colors
            if len(start_color) == len(end_color):
                pixel = tuple(int(start + (end - start) * ratio) 
                             for start, end in zip(start_color, end_color))
            else:
                pixel = start_color
            
            row.append(pixel)
        image.append(row)
    
    return image

def apply_simple_filter(pixel_data, filter_type='invert'):
    """Apply simple filters to pixel data"""
    filtered = []
    
    for row in pixel_data:
        filtered_row = []
        for pixel in row:
            if filter_type == 'invert':
                if isinstance(pixel, (tuple, list)):
                    # Invert RGB/RGBA
                    if len(pixel) == 3:
                        new_pixel = (255 - pixel[0], 255 - pixel[1], 255 - pixel[2])
                    else:  # RGBA
                        new_pixel = (255 - pixel[0], 255 - pixel[1], 255 - pixel[2], pixel[3])
                else:
                    # Invert grayscale
                    new_pixel = 255 - pixel
            elif filter_type == 'brighten':
                if isinstance(pixel, (tuple, list)):
                    new_pixel = tuple(min(255, p + 50) for p in pixel)
                else:
                    new_pixel = min(255, pixel + 50)
            elif filter_type == 'darken':
                if isinstance(pixel, (tuple, list)):
                    new_pixel = tuple(max(0, p - 50) for p in pixel)
                else:
                    new_pixel = max(0, pixel - 50)
            else:
                new_pixel = pixel
            
            filtered_row.append(new_pixel)
        filtered.append(filtered_row)
    
    return filtered

def combine_images_horizontally(img1, img2):
    """Combine two images side by side"""
    if len(img1) != len(img2):
        min_height = min(len(img1), len(img2))
        img1 = img1[:min_height]
        img2 = img2[:min_height]
    
    combined = []
    for i in range(len(img1)):
        combined_row = img1[i] + img2[i]
        combined.append(combined_row)
    
    return combined

# Demonstration of advanced features
print("=== Advanced PNG Writing Examples ===\n")

# Create some example images
print("Creating example images...")

# 1. Solid color images
red_square = create_solid_color_image(60, 60, (255, 0, 0))
blue_circle_data = []
for y in range(60):
    row = []
    for x in range(60):
        # Create a circle
        center_x, center_y = 30, 30
        distance = ((x - center_x) ** 2 + (y - center_y) ** 2) ** 0.5
        if distance <= 25:
            row.append((0, 0, 255, 255))  # Blue with full alpha
        else:
            row.append((255, 255, 255, 0))  # Transparent
    blue_circle_data.append(row)

# 2. Gradient images
horizontal_gradient = create_gradient_image(100, 60, (255, 0, 0), (0, 255, 0), 'horizontal')
vertical_gradient = create_gradient_image(100, 60, (0, 0, 255), (255, 255, 0), 'vertical')
diagonal_gradient = create_gradient_image(100, 60, (255, 0, 255), (0, 255, 255), 'diagonal')

# Save example images
write_png_rgb(red_square, "example_red_square.png")
write_png_rgba(blue_circle_data, "example_blue_circle.png")
write_png_rgb(horizontal_gradient, "example_gradient_h.png")
write_png_rgb(vertical_gradient, "example_gradient_v.png")
write_png_rgb(diagonal_gradient, "example_gradient_d.png")

# 3. Apply filters if we have original image data
if 'pixel_data' in globals() and pixel_data:
    print("Applying filters to original image...")
    
    # Apply different filters
    inverted = apply_simple_filter(pixel_data, 'invert')
    brightened = apply_simple_filter(pixel_data, 'brighten')
    darkened = apply_simple_filter(pixel_data, 'darken')
    
    # Save filtered versions
    write_png_rgb(inverted, "filtered_inverted.png")
    write_png_rgb(brightened, "filtered_brightened.png")
    write_png_rgb(darkened, "filtered_darkened.png")
    
    # Create a comparison image
    if len(pixel_data[0]) <= 200:  # Only if image is not too wide
        comparison = combine_images_horizontally(pixel_data, inverted)
        write_png_rgb(comparison, "comparison_original_inverted.png")

print("\n=== Advanced examples completed! ===")
print("Example files created:")
print("- example_red_square.png")
print("- example_blue_circle.png (with transparency)")
print("- example_gradient_h.png (horizontal gradient)")
print("- example_gradient_v.png (vertical gradient)")
print("- example_gradient_d.png (diagonal gradient)")

if 'pixel_data' in globals() and pixel_data:
    print("- filtered_inverted.png")
    print("- filtered_brightened.png")  
    print("- filtered_darkened.png")
    if len(pixel_data[0]) <= 200:
        print("- comparison_original_inverted.png")

print(f"\nAll PNG files are saved in the current directory!")
print(f"You can now create, modify, and save PNG images using only Python built-ins!")

=== Advanced PNG Writing Examples ===

Creating example images...
PNG file saved: example_red_square.png
Dimensions: 60 x 60
Color type: 2, Bit depth: 8
PNG file saved: example_blue_circle.png
Dimensions: 60 x 60
Color type: 6, Bit depth: 8
PNG file saved: example_gradient_h.png
Dimensions: 100 x 60
Color type: 2, Bit depth: 8
PNG file saved: example_gradient_v.png
Dimensions: 100 x 60
Color type: 2, Bit depth: 8
PNG file saved: example_gradient_d.png
Dimensions: 100 x 60
Color type: 2, Bit depth: 8
Applying filters to original image...
PNG file saved: filtered_inverted.png
Dimensions: 400 x 400
Color type: 2, Bit depth: 8
PNG file saved: filtered_brightened.png
Dimensions: 400 x 400
Color type: 2, Bit depth: 8
PNG file saved: filtered_inverted.png
Dimensions: 400 x 400
Color type: 2, Bit depth: 8
PNG file saved: filtered_brightened.png
Dimensions: 400 x 400
Color type: 2, Bit depth: 8
PNG file saved: filtered_darkened.png
Dimensions: 400 x 400
Color type: 2, Bit depth: 8

=== Advanced